# Demo 7: Recursive Fractal Peeling

This notebook demonstrates the Recursive Fractal Peeling algorithm for lossless data compression.

**Key Concepts:**
- Resfrac invariant: ρ(x) = σ(r) / σ(x) measures predictability
- Autoregressive pattern extraction with residual analysis
- Recursive tree structure for compression
- Lossless reconstruction guarantee: Ψ(Φ(x)) = x

Run all cells to see outputs.

In [ ]:
import numpy as np
from workbench import (
    FractalPeeler,
    resfrac_score,
    compress,
    decompress,
    compression_ratio,
    tree_statistics,
    visualize_tree
)

print('=' * 70)
print('DEMO 7: RECURSIVE FRACTAL PEELING')
print('=' * 70)

## Part 1: Resfrac Score - Measuring Predictability

In [ ]:
# Test different signal types
n = 200

# Highly structured signal (sine wave)
structured = np.sin(np.linspace(0, 4*np.pi, n))
rho_structured = resfrac_score(structured, order=3)

# Random noise
random = np.random.randn(n)
rho_random = resfrac_score(random, order=3)

# Mixed signal
mixed = np.sin(np.linspace(0, 4*np.pi, n)) + 0.3 * np.random.randn(n)
rho_mixed = resfrac_score(mixed, order=3)

print('\nResfrac Scores (ρ ∈ [0,1], lower = more predictable):')
print(f'  Sine wave:      ρ = {rho_structured:.4f} (highly structured)')
print(f'  Random noise:   ρ = {rho_random:.4f} (unpredictable)')
print(f'  Sine + noise:   ρ = {rho_mixed:.4f} (mixed)')

## Part 2: Simple Compression Example

In [ ]:
# Create a structured signal with some complexity
t = np.linspace(0, 10, 500)
signal = np.sin(2*np.pi*t) + 0.5*np.sin(6*np.pi*t) + 0.2*np.random.randn(len(t))

print(f'\nOriginal signal: n = {len(signal)}')
print(f'Resfrac score: ρ = {resfrac_score(signal):.4f}')

# Compress
peeler = FractalPeeler(order=4, max_depth=5)
tree = peeler.compress(signal)

# Get statistics
stats = peeler.tree_stats(tree)
ratio = peeler.compression_ratio(tree, len(signal))

print(f'\nCompression Results:')
print(f'  Tree depth:     {stats["max_depth"]}')
print(f'  Internal nodes: {stats["num_nodes"]}')
print(f'  Leaf nodes:     {stats["num_leaves"]}')
print(f'  Compression:    {ratio:.2f}x')
print(f'  Storage:        {stats["total_size_bytes"]} bytes')

## Part 3: Lossless Reconstruction

In [ ]:
# Decompress
reconstructed = peeler.decompress(tree)

# Verify lossless property
max_error = np.max(np.abs(signal - reconstructed))
mean_error = np.mean(np.abs(signal - reconstructed))

print(f'\nReconstruction Quality:')
print(f'  Max error:  {max_error:.2e}')
print(f'  Mean error: {mean_error:.2e}')
print(f'  Lossless:   {max_error < 1e-10}')

if max_error < 1e-10:
    print('\n✓ Perfect reconstruction: Ψ(Φ(x)) = x')
else:
    print(f'\n⚠ Reconstruction error: {max_error:.2e}')

## Part 4: Tree Visualization

In [ ]:
# Visualize compression tree structure
print('\nCompression Tree Structure:')
print(peeler.visualize(tree))

print('\nTree Interpretation:')
print('  NODE: Internal node with AR model + residuals')
print('  LEAF: Terminal node with raw data')
print('  ρ:    Resfrac score (predictability)')
print('  Δρ:   Improvement from pattern extraction')

## Part 5: Comparison Across Signal Types

In [ ]:
# Test on different signal types
n = 300
peeler = FractalPeeler(order=3, max_depth=8)

signals = {
    'Sine': np.sin(np.linspace(0, 8*np.pi, n)),
    'Polynomial': np.polyval([0.001, -0.1, 2, 10], np.arange(n)),
    'Exponential': np.exp(np.linspace(0, 2, n)),
    'Chirp': np.sin(np.linspace(0, 20*np.pi, n) * np.linspace(1, 3, n)),
    'Random': np.random.randn(n)
}

print('\nCompression Performance by Signal Type:')
print('-' * 70)
print(f'{"Signal":<15} {"ρ":<8} {"Depth":<8} {"Nodes":<8} {"Ratio":<8}')
print('-' * 70)

for name, sig in signals.items():
    rho = resfrac_score(sig)
    tree = peeler.compress(sig)
    stats = peeler.tree_stats(tree)
    ratio = peeler.compression_ratio(tree, len(sig))
    
    print(f'{name:<15} {rho:<8.4f} {stats["max_depth"]:<8} '
          f'{stats["num_nodes"]:<8} {ratio:<8.2f}x')

print('-' * 70)

## Part 6: Advanced - Tuning Parameters

In [ ]:
# Test different AR orders
signal = np.sin(np.linspace(0, 10*np.pi, 400)) + 0.1*np.random.randn(400)

print('\nEffect of AR Model Order:')
print('-' * 60)
print(f'{"Order":<10} {"Depth":<10} {"Nodes":<10} {"Ratio":<10}')
print('-' * 60)

for order in [2, 3, 5, 8]:
    peeler = FractalPeeler(order=order, max_depth=8)
    tree = peeler.compress(signal)
    stats = peeler.tree_stats(tree)
    ratio = peeler.compression_ratio(tree, len(signal))
    
    print(f'{order:<10} {stats["max_depth"]:<10} '
          f'{stats["num_nodes"]:<10} {ratio:<10.2f}x')

print('-' * 60)
print('\nInsight: Higher order captures more complex patterns but')
print('         increases model size. Optimal order depends on signal.')

## Part 7: Integration with Workbench Pipeline

In [ ]:
# Combine with spectral scoring and holographic refinement
from workbench import (
    ZetaFiducials,
    SpectralScorer,
    phase_retrieve_hilbert,
    normalize_signal
)

# Create complex signal
t = np.linspace(0, 20, 1000)
signal = (np.sin(2*np.pi*t) + 0.5*np.sin(6*np.pi*t) + 
          0.3*np.sin(10*np.pi*t) + 0.2*np.random.randn(len(t)))

print('\nIntegrated Pipeline: Spectral → Holographic → Fractal Peeling')
print('=' * 70)

# Step 1: Spectral analysis
zeros = ZetaFiducials.get_standard(10)
scorer = SpectralScorer(zeros)
indices = np.arange(len(signal))
spectral_scores = scorer.compute_scores(indices, mode='real')
print(f'1. Spectral scoring:   mean = {np.mean(spectral_scores):.4f}')

# Step 2: Phase retrieval
envelope, phase_var = phase_retrieve_hilbert(signal)
print(f'2. Phase retrieval:    PV = {phase_var:.4f}')

# Step 3: Fractal peeling compression
peeler = FractalPeeler(order=4, max_depth=6)
tree = peeler.compress(signal)
stats = peeler.tree_stats(tree)
ratio = peeler.compression_ratio(tree, len(signal))
print(f'3. Fractal peeling:    {ratio:.2f}x compression')

# Verify reconstruction
reconstructed = peeler.decompress(tree)
error = np.max(np.abs(signal - reconstructed))
print(f'4. Reconstruction:     error = {error:.2e}')

print('=' * 70)
print('✓ Pipeline complete: Lossless compression with structure analysis')

## Summary

**Recursive Fractal Peeling** provides:
- **Lossless compression** via AR models + residuals
- **Structure detection** through resfrac scores
- **Adaptive depth** based on signal predictability
- **Integration** with spectral and holographic tools

**Key Parameters:**
- `order`: AR model complexity (2-8 typical)
- `noise_threshold`: Stop when ρ > 0.95 (random)
- `improvement_threshold`: Stop when Δρ < 0.01
- `max_depth`: Limit recursion (5-10 typical)

**Use Cases:**
- Compress structured time series
- Analyze signal predictability
- Extract hierarchical patterns
- Combine with spectral/holographic processing